In [ ]:
!pip install r2r -q

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [1]:
from r2r import R2RAsyncClient

client = R2RAsyncClient()

In [2]:
query = "What makes Meta AI’s LLaMA models stand out?"

reference = "Meta AI’s LLaMA models stand out for being open-source, supporting innovation and experimentation due to their accessibility and strong performance."

search_settings = {
        "limit": 2,
        "graph_settings": {"enabled": False, "limit": 2},
    }

response = await client.retrieval.rag(
    query=query,
    search_settings=search_settings
)

In [3]:
response.results.generated_answer

'Meta AI’s LLaMA models stand out due to their open-source nature, which supports innovation and experimentation by making high-quality models accessible to researchers and developers without the need for expensive resources [1], [2]. This democratization of AI development fosters collaboration across industries, allowing a broader range of individuals and organizations to contribute to and benefit from advanced AI technologies [2].'

In [ ]:
response.results.search_results

In [ ]:

chunks = [
    "OpenAI is one of the most recognized names in the large language model space, known for its GPT series of models. These models excel at generating human-like text and performing tasks like creative writing, answering questions, and summarizing content. GPT-4, their latest release, has set benchmarks in understanding context and delivering detailed responses.",
    "Anthropic is well-known for its Claude series of language models, designed with a strong focus on safety and ethical AI behavior. Claude is particularly praised for its ability to follow complex instructions and generate text that aligns closely with user intent.",
    "DeepMind, a division of Google, is recognized for its cutting-edge Gemini models, which are integrated into various Google products like Bard and Workspace tools. These models are renowned for their conversational abilities and their capacity to handle complex, multi-turn dialogues.",
    "Meta AI is best known for its LLaMA (Large Language Model Meta AI) series, which has been made open-source for researchers and developers. LLaMA models are praised for their ability to support innovation and experimentation due to their accessibility and strong performance.",
    "Meta AI with it's LLaMA models aims to democratize AI development by making high-quality models available for free, fostering collaboration across industries. Their open-source approach has been a game-changer for researchers without access to expensive resources.",
    "Microsoft’s Azure AI platform is famous for integrating OpenAI’s GPT models, enabling businesses to use these advanced models in a scalable and secure cloud environment. Azure AI powers applications like Copilot in Office 365, helping users draft emails, generate summaries, and more.",
    "Amazon’s Bedrock platform is recognized for providing access to various language models, including its own models and third-party ones like Anthropic’s Claude and AI21’s Jurassic. Bedrock is especially valued for its flexibility, allowing users to choose models based on their specific needs.",
    "Cohere is well-known for its language models tailored for business use, excelling in tasks like search, summarization, and customer support. Their models are recognized for being efficient, cost-effective, and easy to integrate into workflows.",
    "AI21 Labs is famous for its Jurassic series of language models, which are highly versatile and capable of handling tasks like content creation and code generation. The Jurassic models stand out for their natural language understanding and ability to generate detailed and coherent responses.",
    "In the rapidly advancing field of artificial intelligence, several companies have made significant contributions with their large language models. Notable players include OpenAI, known for its GPT Series (including GPT-4); Anthropic, which offers the Claude Series; Google DeepMind with its Gemini Models; Meta AI, recognized for its LLaMA Series; Microsoft Azure AI, which integrates OpenAI’s GPT Models; Amazon AWS (Bedrock), providing access to various models including Claude (Anthropic) and Jurassic (AI21 Labs); Cohere, which offers its own models tailored for business use; and AI21 Labs, known for its Jurassic Series. These companies are shaping the landscape of AI by providing powerful models with diverse capabilities.",
]

ingest_response = client.documents.create(
    chunks=chunks,
)

In [4]:
questions = [
    "Who are the major players in the large language model space?",
    "What is Microsoft’s Azure AI platform known for?",
    "What kind of models does Cohere provide?",
]

references = [
    "The major players include OpenAI (GPT Series), Anthropic (Claude Series), Google DeepMind (Gemini Models), Meta AI (LLaMA Series), Microsoft Azure AI (integrating GPT Models), Amazon AWS (Bedrock with Claude and Jurassic), Cohere (business-focused models), and AI21 Labs (Jurassic Series).",
    "Microsoft’s Azure AI platform is known for integrating OpenAI’s GPT models, enabling businesses to use these models in a scalable and secure cloud environment.",
    "Cohere provides language models tailored for business use, excelling in tasks like search, summarization, and customer support.",
]

In [5]:
from ragas.dataset_schema import EvaluationDataset

sample_list = []
for i in range(len(questions)):
	sample = {
		"user_input":questions[i],
		"reference":references[i],
		}
	sample_list.append(sample)
		
eval_dataset = EvaluationDataset.from_list(sample_list)
eval_dataset

/Users/nexus/miniconda3/envs/rags2riches/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EvaluationDataset(features=['user_input', 'reference'], len=3)

In [ ]:
from ragas.metrics import AnswerRelevancy, ContextPrecision, Faithfulness
from r2r_new import evaluate
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper

llm = ChatOpenAI(model="gpt-4o-mini")
evaluator_llm = LangchainLLMWrapper(llm)
ragas_metrics = [AnswerRelevancy(), ContextPrecision(), Faithfulness()]

search_settings = {
    "limit": 2,
    "graph_settings": {"enabled": False, "limit": 2},
}

result = evaluate(
    r2r_client=client,
    search_settings=search_settings,
    dataset=eval_dataset,
    llm=evaluator_llm,
    metrics=ragas_metrics,
)
result

Evaluating: 100%|██████████| 9/9 [00:15<00:00,  1.76s/it]


In [7]:
result.to_pandas()

,user_input,retrieved_contexts,response,reference,answer_relevancy,context_precision,faithfulness
0,Who are the major players in the large languag...,[In the rapidly advancing field of artificial ...,The major players in the large language model ...,"The major players include OpenAI (GPT Series),...",1.000000,1.0,1.000000
1,What is Microsoft’s Azure AI platform known for?,[Microsoft’s Azure AI platform is famous for i...,Microsoft’s Azure AI platform is known for int...,Microsoft’s Azure AI platform is known for int...,0.948908,1.0,0.833333
2,What kind of models does Cohere provide?,[Cohere is well-known for its language models ...,Cohere provides language models tailored for b...,Cohere provides language models tailored for b...,0.901376,1.0,1.000000
